# Query With Filtered IDs

Chroma now supports passing a list of IDs at query time to shrink the search space, and allowing for better interoperability with other databases.

This notebook will go over an example using sqlite as a primary database, joining tables to find matching IDs to then compute semantic search across.

In [61]:
! pip install chromadb --quiet
! pip install pandas --quiet
import chromadb
import sqlite3
from datetime import datetime, timedelta
from chromadb.api import ClientAPI
import random
import pandas as pd


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Setup Databases and Sample Data

This is a generated dataset of 100 products and descriptions that fall into one of 5 categories: Electronics, Clothing, Home, Books, Sports. It creates 2 tables in sqlite, one for the categories and the other for the product, along with necessary metadata and the foreign key referencing the category.

While Chroma is capable of metadata storage and filtering which would also solve this case, this is a hypothetical scenario, where you can imagine a user did not want to double store their data, and rather only wanted to store unstructured text data with Chroma.

### Sample Data

In [62]:
product_names = {
    1: [  # Electronics
            "UltraView 55\" 4K Smart TV", "NoiseShield Pro Headphones", "FocusLens X3 DSLR Camera",
            "SoundSphere Portable Speaker", "TitanForce Gaming Laptop", "PowerPad Wireless Charger",
            "GuardianEye Security Camera", "CodeBot Learning Kit", "VitalTrack Fitness Watch",
            "SpeedDrive 1TB SSD", "TactileType Mechanical Keyboard", "GameStation Pro Console",
            "FlexScreen Foldable Phone", "VinylTech Bluetooth Record Player", "ClimateIQ Smart Thermostat",
            "VoicePro Streaming Microphone", "ImmersionVR Headset", "SkyCapture Mini Drone",
            "SmartChill Connected Refrigerator", "SolarCharge Power Bank"
        ],
        2: [  # Clothing
            "Alpine Trek Waterproof Boots", "EcoStitch Organic T-Shirt", "Milano Executive Briefcase",
            "CloudPack Down Jacket", "AlpacaSoft Merino Sweater", "FlexFit Performance Leggings",
            "EcoShade Bamboo Sunglasses", "Heritage Selvedge Jeans", "UrbanSafari Crossbody Bag",
            "JetFresh Travel Shirt", "StreetCanvas Limited Sneakers", "SilkDream Pajama Set",
            "CityTrek Anti-Theft Backpack", "MongolianMist Cashmere Scarf", "TrailBlazer Running Shoes",
            "SlimFold RFID Wallet", "OceanRecycle Swimwear", "SilverStory Charm Bracelet",
            "ClimateSheet Bedding Set", "VoyageWheel Smart Luggage"
        ],
        3: [  # Home
            "BaristaPro Espresso Machine", "CrispWave Air Fryer", "LuxuryRest Egyptian Cotton Sheets",
            "ChromaGlow Smart Bulbs", "AzurCoast Ceramic Dinnerware", "PowerSweep Cordless Vacuum",
            "SimmerMaster Slow Cooker", "CloudRest Memory Foam Mattress", "EcoCut Bamboo Cutting Board",
            "SecureTouch Smart Lock", "PureCoat Ceramic Cookware", "HydroSense Plant Watering System",
            "NordicStyle Coffee Table", "PureBreeze HEPA Air Purifier", "DreamWeight Therapy Blanket",
            "GreenThumb Indoor Herb Garden", "HeritageCook Enameled Dutch Oven", "TouchFlow Kitchen Faucet",
            "ArtisanWeave Wool Rug", "FrostCube Countertop Ice Maker"
        ],
        4: [  # Books
            "Untold: The Icon's Journey", "CulinaryVision AR Cookbook", "Neon Future: Graphic Novel",
            "LinguaLearn AI Software", "Vintage Vinyl Collection: Jazz Classics", "CodeTales: Interactive Kids Book",
            "EarthSolutions Documentary Series", "Present Mind: Meditation Guide", "Declassified: Historical Fiction",
            "CreativeStudio Pro Software", "MindScape Psychology Series", "Atlas of Unexplored Worlds",
            "Financial Freedom Masterclass", "The Complete Sourdough Manual", "History's Mysteries Collection",
            "Digital Photography Masterclass", "Science Explained Encyclopedia", "Classic Literature Anthology",
            "Learn Chess Strategy Guide", "Home Renovation Blueprint"
        ],
        5: [  # Sports
            "CarbonFlight Mountain Bike", "WildernessCook Camping Stove", "EarthGrip Yoga Mat",
            "SkyLite Backpacking Tent", "TrackElite GPS Sports Watch", "PortaYak Collapsible Kayak",
            "PowerFlex Adjustable Dumbbells", "ArcticShield Sleeping Bag", "SmartShot Basketball",
            "SolarGlow Camping Lantern", "OceanRider Custom Surfboard", "ClimbOn Premium Chalk",
            "PureFilter Portable Water Filter", "ProStrike Custom Golf Clubs", "StormShield Rain Jacket",
            "SwingTech Smart Tennis Racket", "ThermoFlask Insulated Bottle", "AquaBeat Floating Speaker",
            "UltraGlide Running Socks", "SurvivalEdge Emergency Kit"
        ]
}
descriptions = {
    1: [
        "Ultra-slim 4K smart TV with HDR support and built-in voice assistant for seamless streaming.",
        "Noise-cancelling wireless headphones with 30-hour battery life and premium audio quality.",
        "Professional-grade DSLR camera with 24.2MP sensor and 4K video recording capabilities.",
        "Compact portable Bluetooth speaker with waterproof design and 360-degree sound.",
        "High-performance gaming laptop featuring the latest graphics card and cooling technology.",
        "Wireless charging pad compatible with all Qi-enabled devices, sleek minimalist design.",
        "Smart home security camera with night vision, motion detection, and cloud storage.",
        "Programmable robot kit for beginners, teaches coding fundamentals through interactive projects.",
        "Fitness smartwatch that tracks heart rate, sleep patterns, and over 20 exercise modes.",
        "Ultra-fast SSD with 1TB storage capacity and lightning-quick data transfer speeds.",
        "Ergonomic mechanical keyboard with RGB backlighting and customizable macro keys.",
        "Next-generation gaming console with ray-tracing support and lightning-fast load times.",
        "Foldable smartphone with dual-screen functionality and advanced multitasking capabilities.",
        "High-fidelity vinyl record player with Bluetooth connectivity and built-in speakers.",
        "Smart thermostat that learns your preferences and optimizes energy consumption.",
        "Professional streaming microphone with studio-quality audio capture and zero latency.",
        "Virtual reality headset with motion tracking and immersive 3D spatial audio.",
        "Compact drone with 4K camera stabilization and 30-minute flight time.",
        "Smart refrigerator with touchscreen display, inventory management, and recipe suggestions.",
        "Solar-powered portable battery pack with multiple charging ports and fast-charge technology.",   
    ],
    2: [
        "Stainless steel espresso machine with precision temperature control and milk frother.",
        "Multi-functional air fryer with 8 cooking presets and dishwasher-safe components.",
        "Egyptian cotton bed sheets with 1000 thread count for hotel-quality comfort.",
        "Smart LED light bulbs that change colors and can be controlled via mobile app.",
        "Handcrafted ceramic dinnerware set inspired by Mediterranean coastal designs.",
        "Cordless stick vacuum with HEPA filtration and up to 40 minutes of runtime.",
        "Programmable slow cooker with digital timer and automatic keep-warm function.",
        "Ergonomic memory foam mattress with cooling gel technology for better sleep.",
        "Eco-friendly bamboo cutting board set with juice groove and non-slip feet.",
        "Smart door lock with fingerprint recognition and temporary access codes.",
        "Non-stick ceramic cookware set free from PFOA and other harmful chemicals.",
        "Automatic plant watering system with soil moisture sensors and app connectivity.",
        "Scandinavian-inspired minimalist wooden coffee table with hidden storage.",
        "High-efficiency air purifier that removes 99.97% of allergens and pollutants.",
        "Luxury weighted blanket that promotes relaxation and deeper sleep.",
        "Smart indoor herb garden with self-watering system and growth lights.",
        "Cast iron Dutch oven with enamel coating, perfect for slow-cooking and baking.",
        "Motion-activated kitchen faucet with temperature memory and water-saving mode.",
        "Handwoven wool area rug made using traditional artisanal techniques.",
        "Compact countertop ice maker producing crystal-clear ice cubes in minutes.",
    ],
    3: [
        "Waterproof hiking boots with vibram soles and Gore-Tex membrane for all-weather trekking.",
        "Sustainably made organic cotton t-shirt dyed using natural plant-based pigments.",
        "Premium Italian leather briefcase with multiple compartments and laptop protection.",
        "Ultralight packable down jacket that compresses into its own pocket for travel.",
        "Ethically sourced merino wool sweater that regulates temperature in all seasons.",
        "Moisture-wicking athletic leggings with four-way stretch and hidden pocket.",
        "Handcrafted polarized sunglasses with sustainable bamboo frames and UV protection.",
        "Classic selvedge denim jeans made on vintage looms for exceptional durability.",
        "Vegan leather crossbody bag with RFID protection and organizational pockets.",
        "Wrinkle-resistant travel dress shirt that stays fresh after long flights.",
        "Limited edition sneakers featuring artwork from emerging urban artists.",
        "Silk pajama set that helps regulate body temperature for better sleep.",
        "Water-resistant canvas backpack with anti-theft features and laptop compartment.",
        "Luxury cashmere scarf ethically sourced from Mongolian mountain goats.",
        "Breathable trail running shoes with responsive cushioning and rock plate protection.",
        "Hand-stitched leather wallet with RFID blocking technology and slim profile.",
        "Reversible swimwear made from recycled ocean plastic and quick-dry fabric.",
        "Customizable charm bracelet crafted from sterling silver with meaningful pendants.",
        "Temperature-regulating bedding that keeps you cool in summer and warm in winter.",
        "Modular travel luggage with built-in USB charging port and silent wheels.",
    ],
    4: [  # Books
        "Limited edition signed autobiography revealing untold stories of a cultural icon.",
        "Immersive cooking cookbook with augmented reality recipes that come to life.",
        "Award-winning graphic novel exploring themes of identity in a dystopian future.",
        "Language learning software that adapts to your progress using AI technology.",
        "Vintage vinyl record collection featuring remastered classic jazz recordings.",
        "Interactive children's book that teaches coding concepts through storytelling.",
        "Documentary film series exploring sustainable solutions to environmental challenges.",
        "Mindfulness meditation guide with accompanying app for daily practice.",
        "Historical fiction novel based on recently declassified government documents.",
        "Digital art creation software with professional-grade tools and tutorials.",
        "Comprehensive psychology series exploring the latest research in human behavior.",
        "Beautifully illustrated atlas featuring unexplored regions and indigenous knowledge.",
        "Interactive financial course teaching investment strategies and wealth building.",
        "Complete guide to sourdough baking with scientific explanations and techniques.",
        "Collection of books examining history's greatest unsolved mysteries and legends.",
        "Digital photography masterclass with video tutorials from renowned photographers.",
        "Encyclopedia explaining complex scientific concepts in accessible language.",
        "Leather-bound anthology of world literature classics with scholarly annotations.",
        "Strategic chess guide written by grandmasters with interactive practice modules.",
        "Comprehensive home renovation manual with step-by-step instructions and plans."
    ],
    5: [
        "Carbon fiber mountain bike with full suspension and hydraulic disc brakes.",
        "Portable camping stove that runs on multiple fuel types for wilderness cooking.",
        "Professional-grade yoga mat made from sustainable natural rubber with perfect grip.",
        "Ultralight backpacking tent that sets up in minutes and withstands harsh weather.",
        "GPS sports watch with advanced training metrics and recovery recommendations.",
        "Collapsible kayak that fits in your trunk yet performs like a rigid model on water.",
        "Adjustable dumbbell set that replaces 15 pairs of weights to save space.",
        "All-season sleeping bag rated for extreme temperatures with water-resistant shell.",
        "Smart basketball that tracks shooting accuracy and provides coaching feedback.",
        "Solar-powered camping lantern with emergency phone charging capability.",
        "Surfboard shaped by renowned craftsmen using sustainable materials and techniques.",
        "Premium rock climbing chalk that maximizes grip while minimizing environmental impact.",
        "Portable water filter that removes 99.9% of bacteria and parasites for safe drinking.",
        "Golf clubs custom-fitted to your swing metrics for improved performance.",
        "Breathable rain jacket with sealed seams and adjustable hood for outdoor adventures.",
        "Smart tennis racket that analyzes your swing and provides improvement tips.",
        "Insulated stainless steel water bottle that keeps drinks cold for 24 hours.",
        "Floating bluetooth speaker designed for pool parties and beach outings.",
        "High-performance running socks that prevent blisters and enhance comfort.",
        "Compact emergency survival kit with essential tools for outdoor safety.",
    ],
}

price_ranges = {
        1: (299.99, 1999.99),    # Electronics
        2: (19.99, 299.99),      # Clothing
        3: (29.99, 499.99),      # Home
        4: (9.99, 99.99),        # Books
        5: (24.99, 899.99)       # Sports
    }

### Instantiate Databases and Insert Data

In [63]:
def setup_databases(conn, client: ClientAPI):
    """Set up SQLite and ChromaDB with sample data."""
    print("Setting up databases...")
    
    # Create SQLite database
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY,
        name TEXT,
        category_id INTEGER,
        price REAL,
        description TEXT,
        created_date TEXT,
        FOREIGN KEY (category_id) REFERENCES categories(id)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS categories (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
    ''')
    
    categories = ['Electronics', 'Clothing', 'Home', 'Books', 'Sports']
    for i, category in enumerate(categories, 1):
        cursor.execute("INSERT INTO categories VALUES (?, ?)", (i, category))
        
    
    # Generate random dates within the last year
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    # Insert products
    product_id = 1
    product_texts = []
    product_ids = []
    for category_id in range(1, 6):
        min_price, max_price = price_ranges[category_id]
        
        for i, product_name in enumerate(product_names[category_id]):
            description = descriptions[category_id][i] if i < len(descriptions[category_id]) else "Product description not available."
            
            # Generate random price within the category's range
            price = round(random.uniform(min_price, max_price), 2)
            
            # Generate random creation date
            random_days = random.randint(0, 365)
            created_date = (start_date + timedelta(days=random_days)).strftime('%Y-%m-%d')
            
            # Insert product into database
            cursor.execute(
                "INSERT INTO products VALUES (?, ?, ?, ?, ?, ?)",
                (product_id, product_name, category_id, price, description, created_date)
            )
            
            product_ids.append(str(product_id))
            product_id += 1
            product_texts.append(f"{product_name}: {description}")
    
    # Commit changes and return connection
    conn.commit()
        
    collection = client.get_or_create_collection(
        name="product_embeddings",
    )
        
    collection.add(
        ids=product_ids,
        documents=product_texts,
    )
    
    print(f"Database setup complete. Added {product_id-1} products across {len(categories)} categories.")
    

In [64]:
conn = sqlite3.connect(':memory:')
client = chromadb.EphemeralClient()

setup_databases(conn, client)

Setting up databases...
Database setup complete. Added 100 products across 5 categories.


In [65]:
def view_sqlite_data(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM categories")
    df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name'])
    print(df)
    cursor.execute("SELECT * FROM products")
    df = pd.DataFrame(cursor.fetchall(), columns=['id', 'name', 'category_id', 'price', 'description', 'created_date'])
    print(df)
    

view_sqlite_data(conn)

   id         name
0   1  Electronics
1   2     Clothing
2   3         Home
3   4        Books
4   5       Sports
     id                           name  category_id    price  \
0     1      UltraView 55" 4K Smart TV            1   395.76   
1     2     NoiseShield Pro Headphones            1   422.67   
2     3       FocusLens X3 DSLR Camera            1   935.73   
3     4   SoundSphere Portable Speaker            1  1886.39   
4     5       TitanForce Gaming Laptop            1  1821.69   
..  ...                            ...          ...      ...   
95   96  SwingTech Smart Tennis Racket            5   819.70   
96   97   ThermoFlask Insulated Bottle            5   442.99   
97   98      AquaBeat Floating Speaker            5   334.21   
98   99       UltraGlide Running Socks            5   721.08   
99  100     SurvivalEdge Emergency Kit            5   435.15   

                                          description created_date  
0   Ultra-slim 4K smart TV with HDR support and 

## Querying using Filtered IDs
We will now use the sqlite to first fetch the list of ids we would want to query on in 2 scenarios.

Scenario 1: The user asks to see eco-friendly products in the Electronics category. We will fetch IDs corresponding to the Electronics category via foreign key, and use the set of IDs to shrink our search space.

Scenario 2: The user wants to find items of a specific price across multiple categories. This will utilize more complex filtering in sqlite to then narrow our search space before doing semantic search.

In [66]:

def demo_filtered_search(conn, chroma_client: ClientAPI):
    cursor = conn.cursor()
    collection = chroma_client.get_collection(
        name="product_embeddings",
    )
    
    # SCENARIO 1: Find eco-friendly products in the 'Electronics' category
    cursor.execute("""
        SELECT p.id 
        FROM products p
        JOIN categories c ON p.category_id = c.id
        WHERE c.name = 'Electronics'
    """)
    
    category_product_ids = [str(row[0]) for row in cursor.fetchall()]
    
    print(f"Found {len(category_product_ids)} products in 'Electronics' category")
    
    # Step 2: Use these IDs to filter ChromaDB search
    query_text = "eco-friendly sustainable product"
    
    results = collection.query(
        query_texts=[query_text],
        ids=category_product_ids,
        n_results=3,
    )
    
    # Step 3: Get full product details from SQLite
    print("\nResults (eco-friendly Electronics products):")
    
    for i, product_id in enumerate(results['ids'][0]):
        cursor.execute("""
            SELECT p.name, c.name, p.price, p.description
            FROM products p
            JOIN categories c ON p.category_id = c.id
            WHERE p.id = ?
        """, (product_id,))
        
        product_name, category_name, price, description = cursor.fetchone()
        print(f"\n{i+1}. {product_name} (${price:.2f}) - {category_name}")
        print(f"   Description: {description}")
        
        
    # SCENARIO 2: Find products within a specific price range across multiple categories
    print("\n\nSCENARIO: Find innovative products between $50-$150 in Home or Electronics categories\n")
    
    # Step 1: Get products matching structured criteria using SQL
    cursor.execute("""
        SELECT p.id 
        FROM products p
        JOIN categories c ON p.category_id = c.id
        WHERE (c.name = 'Home' OR c.name = 'Electronics')
        AND p.price BETWEEN 50.0 AND 150.0
    """)
    
    filtered_product_ids = [str(row[0]) for row in cursor.fetchall()]
    
    print(f"Found {len(filtered_product_ids)} products matching price range and categories")
    
    # Step 2: Use these IDs to filter ChromaDB search
    query_text = "innovative modern design"
    
    results = collection.query(
        query_texts=[query_text],
        ids=filtered_product_ids,
        n_results=3,
    )
    
    # Step 3: Get full product details from SQLite
    print("\nResults (innovative products in filtered categories and price range):")
    
    for i, product_id in enumerate(results['ids'][0]):
        cursor.execute("""
            SELECT p.name, c.name, p.price, p.description
            FROM products p
            JOIN categories c ON p.category_id = c.id
            WHERE p.id = ?
        """, (product_id,))
        
        product_name, category_name, price, description = cursor.fetchone()
        print(f"\n{i+1}. {product_name} (${price:.2f}) - {category_name}")
        print(f"   Description: {description}")

In [67]:
demo_filtered_search(conn, client)

Found 20 products in 'Electronics' category

Results (eco-friendly Electronics products):

1. SolarCharge Power Bank ($1899.14) - Electronics
   Description: Solar-powered portable battery pack with multiple charging ports and fast-charge technology.

2. VoicePro Streaming Microphone ($1051.11) - Electronics
   Description: Professional streaming microphone with studio-quality audio capture and zero latency.

3. FocusLens X3 DSLR Camera ($935.73) - Electronics
   Description: Professional-grade DSLR camera with 24.2MP sensor and 4K video recording capabilities.


SCENARIO: Find innovative products between $50-$150 in Home or Electronics categories

Found 4 products matching price range and categories

Results (innovative products in filtered categories and price range):

1. GreenThumb Indoor Herb Garden ($112.80) - Home
   Description: Hand-stitched leather wallet with RFID blocking technology and slim profile.

2. PowerSweep Cordless Vacuum ($60.34) - Home
   Description: Moisture-wic